In [1]:
import time

from data_stream import StreamReceiver

from simulation.webots.controllers.ardupilot_vehicle_controller.drone_data import RangefinderData, CameraData, FDMData, GimbalAxisData, GimbalData, DroneData
from mavlink.mavlink import MAVLinkController, DataAcquisitionThread
from mavlink.mavlink.processor import DataProcessor, GimbalProcessor, GlobalPositionProcessor, AttitudeProcessor, LocalPositionProcessor
from math import atan, tan, radians, degrees
import time
import numpy
from mavlink.mavlink.data import Gimbal
import cv2
from data_stream import StreamReceiver
from simulation.webots.controllers.ardupilot_vehicle_controller.drone_data import RangefinderData, CameraData, FDMData, GimbalAxisData, GimbalData, DroneData
import math
from matplotlib import pyplot as plt
from ultralytics import YOLO
import random
from deep_sort.deep_sort.tracker import Tracker
from deep_sort.deep_sort.deep.extractor import Extractor
from deep_sort.deep_sort.deep.configuration import ResNetConfiguration
from deep_sort.deep_sort.deep.weights import RESNET18_WEIGHTS


mavlink_connection = MAVLinkController("udp:0.0.0.0:14550")

attitude_processor = AttitudeProcessor()
global_position_processor = GlobalPositionProcessor()
local_position_processor = LocalPositionProcessor()
gimbal_processor = GimbalProcessor()

acquisition_thread = DataAcquisitionThread(mavlink_connection, [attitude_processor, global_position_processor, local_position_processor, gimbal_processor])
acquisition_thread.start()


host = "192.168.0.107"
port = 5588
stream_receiver = StreamReceiver(host, port)

In [71]:
latest_data = gimbal_processor.queue.get_latest()
if latest_data:
    print("GIMBAL", [math.degrees(axis) for axis in latest_data.quaternion.to_euler()])

mavlink_connection.gimbal.set_angles(
    roll=0,
    pitch=-10,
    yaw=0
)

In [72]:
data = stream_receiver.get_data()
drone_data = DroneData.from_json(data)
camera_frame = drone_data.camera.frame

global_position = global_position_processor.queue.get_latest()
gimbal = gimbal_processor.queue.get_latest()

plt.imshow(camera_frame)

In [6]:
model = YOLO("yolov8x.pt")

colors = [(
    random.randint(0, 255),
    random.randint(0, 255),
    random.randint(0, 255)) for j in range(10)
]

In [73]:
results = model.predict(
    camera_frame,
    verbose=False
)
result = results[0]

yolo_frame = camera_frame.copy()
detections = []
class_list = [0, 2]
for i, result in enumerate(result.boxes.data.tolist()):    
    x1, y1, x2, y2, score, class_id = result
    x1 = int(x1)
    x2 = int(x2)
    y1 = int(y1)
    y2 = int(y2)
    class_id = int(class_id)    
    
    print([x1, y1, x2, y2, score, class_id, model.names[class_id]])
    
    if class_id in class_list:
        detections.append([x1, y1, x2, y2, score, class_id])
    else:
        continue
    
    color = colors[i % len(colors)]

    cv2.rectangle(
        yolo_frame,
        (int(x1), int(y1)),
        (int(x2), int(y2)),
        color,
        3
    )
    
    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)
    
    cv2.circle(
        yolo_frame,
        (cx, cy),
        3,
        color,
        -1
    )
    
    cv2.putText(
        yolo_frame,
        f"{model.names[class_id]} {round(score * 100, 0)}%",
        (cx + 10, cy),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 0, 255),
        2
    )
    
plt.imshow(yolo_frame)

In [105]:
import rasterio
from rasterio.enums import Resampling
from rasterio.warp import transform


class GEOSpatial:
    def __init__(self, file_path):
        self.file_path = file_path
        self.src = rasterio.open(self.file_path)
        self.resolution = self._estimate_resolution()
        
        self.elevation_cache = {}
    
    def _estimate_resolution(self):
        latitude_resolution = (self.src.bounds.top - self.src.bounds.bottom) / self.src.height
        longitude_resolution = (self.src.bounds.right - self.src.bounds.left) / self.src.width    
        
        return max(latitude_resolution, longitude_resolution)
    
    def round_location(self, latitude, longitude):
        decimal_places = -int(math.floor(math.log10(self.resolution)))

        latitude_round = round(round(latitude / self.resolution) * self.resolution, decimal_places)
        longitude_round = round(round(longitude / self.resolution) * self.resolution, decimal_places)
        
        return latitude_round, longitude_round        
    
    def find_elevation(self, latitude, longitude):
        latitude_key, longitude_key = self.round_location(latitude, longitude)
        
        if (latitude_key, longitude_key) in self.elevation_cache:
            return self.elevation_cache[(latitude_key, longitude_key)]
        
        transformed_point = transform(
            {"init": "epsg:4326"}, 
            self.src.crs,  
            [longitude], [latitude]
        )
        
        row, col = self.src.index(transformed_point[0][0], transformed_point[1][0])
        
        elevation = self.src.read(
            1, 
            window=rasterio.windows.Window(col, row, 1, 1), 
            resampling=Resampling.nearest
        )
        
        elevation_value = elevation[0][0]
        
        self.elevation_cache[(latitude_key, longitude_key)] = elevation_value
        
        return elevation_value    


file_path = "S36E149.hgt"
geospatial = GEOSpatial(file_path)


def find_target_location(global_position, gimbal, offset=None):    
    roll, pitch, yaw = gimbal.quaternion.to_euler()
    heading = math.radians(global_position.heading)  
    yaw = yaw + heading
    
    if offset:
        offset_roll, offset_pitch, offset_yaw = offset
        
        roll += offset_roll
        pitch += offset_pitch
        yaw += offset_yaw
    
    direction_vector = numpy.array([
        math.cos(pitch) * math.sin(yaw),
        math.cos(pitch) * math.cos(yaw),
        math.sin(pitch)
    ])
    
    distance_to_ground = global_position.relative_altitude / -direction_vector[2]
    
    target_enu = direction_vector * distance_to_ground
    
    earth_radius = 6371000
    latitude_change = target_enu[1] / earth_radius
    longitude_change = target_enu[0] / (earth_radius * math.cos(math.radians(global_position.latitude)))
    
    target_latitude = global_position.latitude + math.degrees(latitude_change)
    target_longitude = global_position.longitude + math.degrees(longitude_change)
    
    return target_latitude, target_longitude


def distance_between_locations(lat1, lon1, alt1, lat2, lon2, alt2):
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    lambda1, lambda2 = math.radians(lon1), math.radians(lon2)
    
    delta_phi = phi2 - phi1
    delta_lambda = lambda2 - lambda1
    delta_alt = alt2 - alt1
    
    a = math.sin(delta_phi / 2.0) ** 2 + math.cos(phi1) * math.cos(phi2) * math.sin(delta_lambda / 2.0) ** 2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1 - a))
    earth_radius = 6371000
    distance = earth_radius * c
    
    total_distance = math.sqrt(distance ** 2 + delta_alt ** 2)
    
    return total_distance


In [104]:
fov_vertical = 2 * atan(tan(drone_data.camera.fov * 0.5) * (drone_data.camera.height / drone_data.camera.width))

def find_center(x1, y1, x2, y2):
    center_x = (x1 + x2) // 2
    center_y = (y1 + y2) // 2
    
    return center_x, center_y


geo_frame = camera_frame.copy()

target_location = []
for i, track in enumerate(detections):
    x1, y1, x2, y2, score, class_id = track
    
    target_center_x, target_center_y = find_center(x1, y1, x2, y2)
    
    image_center_x, image_center_y = find_center(0, 0, drone_data.camera.width, drone_data.camera.height)
    
    offset_x = target_center_x - image_center_x
    offset_y = target_center_y - image_center_y
    
    offset_yaw = (offset_x / drone_data.camera.width) * drone_data.camera.fov
    offset_pitch = -(offset_y / drone_data.camera.height) * fov_vertical    
    
    target_latitude, target_longitude = find_target_location(
        global_position, 
        gimbal, 
        offset=[0, offset_pitch, offset_yaw]
    )
    target_altitude = geospatial.find_elevation(target_latitude, target_longitude)
    
    target_location.append([i, class_id, target_latitude, target_longitude, target_altitude])
    
    distance = distance_between_locations(
        target_latitude, target_longitude, target_altitude, 
        global_position.latitude, global_position.longitude, global_position.altitude
    )
    
    color = colors[i % len(colors)]

    cx = int((x1 + x2) / 2)
    cy = int((y1 + y2) / 2)

    cv2.circle(
        geo_frame,
        (cx, cy),
        3,
        color,
        -1
    )

    cv2.putText(
        geo_frame,
        f"ID: {i}",
        (cx + 10, cy),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 0, 255),
        2
    )
    cv2.putText(
        geo_frame,
        f"DISTANCE: {round(distance, 0)} m",
        (cx + 10, cy + 15),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 0, 255),
        2
    )
    cv2.putText(
        geo_frame,
        f"LAT: {round(target_latitude, 6)}",
        (cx + 10, cy + 30),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 0, 255),
        2
    )
    cv2.putText(
        geo_frame,
        f"LON: {round(target_longitude, 6)}",
        (cx + 10, cy + 45),
        cv2.FONT_HERSHEY_SIMPLEX,
        0.5,
        (0, 0, 255),
        2
    )
    
    
    
plt.imshow(geo_frame)

In [107]:
print(global_position)
print(gimbal.quaternion.to_euler())
print(target_location)

In [108]:
import folium


folium_map = folium.Map(location=[global_position.latitude, global_position.longitude], zoom_start=18)


basemaps = {
    "OpenStreetMap": folium.TileLayer(
        tiles = "OpenStreetMap",
        name = "OpenStreetMap"
    ),
    "Google Maps": folium.TileLayer(
        tiles = "https://mt1.google.com/vt/lyrs=m&x={x}&y={y}&z={z}",
        attr = "Google",
        name = "Google Maps",
        overlay = True,
        control = True
    ),
    "Google Satellite": folium.TileLayer(
        tiles = "https://mt1.google.com/vt/lyrs=s&x={x}&y={y}&z={z}",
        attr = "Google",
        name = "Google Satellite",
        overlay = True,
        control = True
    ),
    "Google Terrain": folium.TileLayer(
        tiles = "https://mt1.google.com/vt/lyrs=p&x={x}&y={y}&z={z}",
        attr = "Google",
        name = "Google Terrain",
        overlay = True,
        control = True
    ),
    "Google Satellite Hybrid": folium.TileLayer(
        tiles = "https://mt1.google.com/vt/lyrs=y&x={x}&y={y}&z={z}",
        attr = "Google",
        name = "Google Satellite",
        overlay = True,
        control = True
    ),
    "Esri Satellite": folium.TileLayer(
        tiles = "https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}",
        attr = "Esri",
        name = "Esri Satellite",
        overlay = True,
        control = True
    )
}

basemaps["Esri Satellite"].add_to(folium_map)


folium.Marker(
    [global_position.latitude, global_position.longitude],
    popup="Drone Location",
    icon=folium.Icon(color="blue", icon="plane"),
    tooltip="Click for more info"
).add_to(folium_map)

for target in target_location:
    idx, class_id, target_latitude, target_longitude, target_altitude = target
    folium_marker = folium.Marker(
        [target_latitude, target_longitude],
        popup=f"Target {idx} - {model.names[class_id]}",
        icon=folium.Icon(color="red", icon="flag"),
        tooltip="Click for more info"
    )
    folium_marker.add_to(folium_map)


folium_map

In [124]:
import math


def calculate_target_point(lat, lon, altitude, relative_altitude, heading, pitch):
    R = 6371000  # Approximate radius of earth in meters
    absolute_altitude = altitude
    
    # Convert degrees to radians
    heading_rad = math.radians(heading)
    pitch_rad = math.radians(pitch)
    
    # Calculate the direction vector components in the Earth-centered Earth-fixed (ECEF) coordinate system
    dx = math.cos(pitch_rad) * math.sin(heading_rad)
    dy = math.cos(pitch_rad) * math.cos(heading_rad)
    dz = math.sin(pitch_rad)
    
    # Start from the camera position
    current_lat = lat
    current_lon = lon
    current_elevation = absolute_altitude
    distance_step = 1  # Step by 0.1 meters
    
    # Loop to find where the vector intersects with the ground
    while True:
        # Move along the vector
        current_lat += (dy * distance_step) / (R * math.pi / 180)
        current_lon += (dx * distance_step) / (R * math.cos(math.radians(current_lat)) * math.pi / 180)
        current_elevation += dz * distance_step
        
        # Check if we are below the ground level
        ground_elevation = geospatial.find_elevation(current_lat, current_lon)
        if current_elevation <= ground_elevation:
            return current_lat, current_lon, ground_elevation
        
        # If the current elevation goes negative too far below sea level, break the loop to avoid infinite execution
        if current_elevation < -1000:
            break
    
    return None  # No ground intersection found within reasonable limits

# Test the function with the provided data
latitude = -35.3633662
longitude = 149.1653247
altitude = 594.06
relative_altitude = 9.99

roll, pitch, yaw = gimbal.quaternion.to_euler()
heading = global_position.heading

target_point = calculate_target_point(latitude, longitude, altitude, relative_altitude, heading, math.degrees(pitch))
print("Target Point (Latitude, Longitude, Elevation):", target_point)


In [125]:
print(distance_between_locations(
    global_position.latitude,
    global_position.longitude,
    global_position.altitude,
    target_point[0], target_point[1], target_point[2]
))